### 1 - Load the data using the .read() function to create a DataFrame.

In [10]:
from pyspark.sql import SparkSession

# Initialisation SparkSession
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

# read
df = spark.read.option("sep", "\t").option("inferSchema", "true").option("header", "false").csv("/data/u.data")

# Ajout noms des colonnes
df = df.withColumnRenamed("_c0", "user_id").withColumnRenamed("_c1", "item_id").withColumnRenamed("_c2", "rating").withColumnRenamed("_c3", "timestamp")

# Affichage des 5 premières lignes
df.show(5)


+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



### 2 - Filter the DataFrame to include only ratings of 1 or 2.

In [11]:
filtered_df = df.filter((df["rating"] == 1) | (df["rating"] == 2))
filtered_df.show(10)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    115|    265|     2|881171488|
|     62|    257|     2|879372434|
|    194|    274|     2|879539794|
|    234|   1184|     2|892079237|
|    291|    118|     2|874833878|
|     95|    546|     2|879196566|
|    102|    768|     2|883748450|
+-------+-------+------+---------+
only showing top 10 rows



In [12]:
filtered_df = df.filter((df.rating == 1) | (df.rating == 2))
filtered_df.show(10)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    115|    265|     2|881171488|
|     62|    257|     2|879372434|
|    194|    274|     2|879539794|
|    234|   1184|     2|892079237|
|    291|    118|     2|874833878|
|     95|    546|     2|879196566|
|    102|    768|     2|883748450|
+-------+-------+------+---------+
only showing top 10 rows



### 3 - Aggregate this data to find users with the highest number of low ratings.

In [13]:
from pyspark.sql import functions as F

In [15]:
# Agréger pour compter le nb de notes faibles par user
low_ratings_users = filtered_df.groupBy("user_id").agg(F.count("rating"))

In [19]:
# trie décroissant
low_ratings_users_desc = low_ratings_users.orderBy(F.desc(F.count("rating")))

# afficher les 5 head
low_ratings_users_desc.show(5)

+-------+-------------+
|user_id|count(rating)|
+-------+-------------+
|    405|          558|
|    181|          380|
|     13|          213|
|    655|          181|
|    774|          160|
+-------+-------------+
only showing top 5 rows



### 4 - Use SQL queries for filtering and aggregating the data.

In [20]:
# vue temporaire pour exécuter des requêtes SQL
df.createOrReplaceTempView("ratings")

In [21]:
query = """
    SELECT user_id, COUNT(rating) AS low_rating_count
    FROM ratings
    WHERE rating IN (1, 2)
    GROUP BY user_id
    ORDER BY low_rating_count DESC
    LIMIT 5
"""

result = spark.sql(query)

result.show()

+-------+----------------+
|user_id|low_rating_count|
+-------+----------------+
|    405|             558|
|    181|             380|
|     13|             213|
|    655|             181|
|    774|             160|
+-------+----------------+



### 5 - For an enhanced visual presentation, apply the asDict() method in your print command.

In [22]:
# Afficher les résultats avec asDict()
print("users ayant le plus grand nombre de notes faibles :")
for row in result.collect():
    print(row.asDict())

users ayant le plus grand nombre de notes faibles :
{'user_id': 405, 'low_rating_count': 558}
{'user_id': 181, 'low_rating_count': 380}
{'user_id': 13, 'low_rating_count': 213}
{'user_id': 655, 'low_rating_count': 181}
{'user_id': 774, 'low_rating_count': 160}


In [23]:
# Arrêt de la SparkSession
spark.stop()